In [1]:
import mne
import signal_metrics
from pathlib import Path
import bids
import pickle
from eeg_fmri_cleaning import GradientRemover
import numpy as np
import os
import eeg_fmri_cleaning.main as cleaner

In [2]:
%%capture
def mne_metrics(filename):
    filename_entities = bids.layout.parse_file_entities(filename.name)

    if os.path.splitext(filename)[1] == '.set':
        raw = mne.io.read_raw_eeglab(filename, preload=True)
        channel_map = cleaner.map_channel_type(raw)
        raw = cleaner.set_channel_types(raw, channel_map)
    elif os.path.splitext(filename)[1] == '.fif':
        raw = mne.io.read_raw_fif(filename, preload=True)
        
    if filename_entities['task'] == 'checker':
        trigger_name = GradientRemover.extract_gradient_trigger_name(raw)
        annotations = raw.annotations
        index_grad_trigger = np.where(annotations.description == trigger_name)
        third_grad_trigger = annotations.onset[index_grad_trigger[0][1]]
        last_grad_trigger = annotations.onset[index_grad_trigger[0][-1]]
        
        raw.crop(tmin=third_grad_trigger, tmax=last_grad_trigger)
    
    raw_time_metrics = signal_metrics.SignalMetrics(raw)
    raw_time_metrics.calculate_time_metrics()

    noise_freq_tagging = signal_metrics.SignalMetrics(raw)
    noise_freq_tagging.calculate_frequency_metrics(frequency_range=(17,20),
                                                                frequency_type="noise")
    ssvep_freq_tagging = signal_metrics.SignalMetrics(raw)
    ssvep_freq_tagging.calculate_frequency_metrics(frequency_range=(11,13),
                                                                frequency_type="signal")
    ecg_epochs = mne.preprocessing.create_ecg_epochs(raw)
    epochs_ecg_metrics = signal_metrics.SignalMetrics(ecg_epochs)
    epochs_ecg_metrics.calculate_time_metrics()
    
    
    filename_path = Path(filename)
    base_filename =  os.path.splitext(filename_path)[0]
    
    for metric_object, metric_object_name in zip([raw_time_metrics, 
                                                  noise_freq_tagging, 
                                                  ssvep_freq_tagging, 
                                                  epochs_ecg_metrics], 
                                                 ['raw_time_metrics', 
                                                  'noise_freq_tagging', 
                                                  'ssvep_freq_tagging', 
                                                  'epochs_ecg_metrics']):

        saving_filename = base_filename + '_' + metric_object_name + '.pkl'
        with open(saving_filename, 'wb') as f:
            pickle.dump(metric_object, f)
        print(f'{saving_filename} saved')




In [ ]:
%%capture
path = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES')
folders = ['BCG',
         'BCG_ASR',
         'GRAD_BCG',
         'GRAD_BCG_ASR']
errors = {}
for folder in folders:
    child_path = path / folder
    for child_child_path in child_path.iterdir():
        for child_child_child_path in child_child_path.iterdir():
            eeg_path = child_child_child_path / 'eeg'
            for file in eeg_path.iterdir():
                if os.path.splitext(file)[1] == '.fif':
                    mne_metrics(file)

In [3]:
%%capture
path = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/')
errors = {}
for file in path.iterdir():
    if os.path.splitext(file)[1] == '.set':
        filename_entities = bids.layout.parse_file_entities(file.name)
        if 'checker' in filename_entities['task']:
            try:
                mne_metrics(file)
            except Exception as e:
                errors[file] = e
                print(f'Error in {file}: {e}')
                continue

In [4]:
errors

{PosixPath('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-15_ses-02_task-checker_run-01_eeg-edf.set'): RuntimeError('Incorrect number of samples (958464 != 8608000), please report this error to MNE-Python developers'),
 PosixPath('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-15_ses-01_task-checker_run-01_eeg-edf.set'): RuntimeError('Incorrect number of samples (524288 != 3760000), please report this error to MNE-Python developers'),
 PosixPath('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-03_ses-01_task-checker_run-01_eeg-edf_2.set'): RuntimeError('Incorrect number of samples (3162112 != 3840000), please report this error to MNE-Python developers'),
 PosixPath('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-14_ses-01_task-checker_run-01_eeg-edf.set'): RuntimeError('Incorrect number of samples (393216 != 4480000), please report this error to MNE-Python developers'),
 PosixPath('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-14_ses-02_task-checker_

In [2]:
raw = mne.io.read_raw_eeglab('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-22_ses-01_task-checker_run-01_eeg-edf.set')
metrics = signal_metrics.SignalMetrics(raw)
metrics.calculate_frequency_metrics(frequency_range=(11,13), frequency_type='signal')
metrics.amplitude.shape

Reading /projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-22_ses-01_task-checker_run-01_eeg-edf.fdt


TypeError: rfft() got an unexpected keyword argument 'axes'

In [4]:
import scipy 
import frequency_analysis
import mne
import matplotlib.pyplot as plt
frequency_range = (11,13)
raw = mne.io.read_raw_eeglab('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-22_ses-01_task-checker_run-01_eeg-edf.set')
array = raw.get_data()
fft1 = scipy.fft.fft(array)
fft2 = (scipy.fft.rfft(array) * 2) / array.shape[1]
spectrum = frequency_analysis.Spectrum()
fft0 = spectrum.calculate_fft(raw)
amplitude = fft0.calculate_amplitude()
amplitude.get_peak_magnitude(frequency_range)
amplitude._set_frequency_of_interest(amplitude.peak_frequency_Hz)
zscore = fft0.copy().calculate_zscore()

Reading /projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-22_ses-01_task-checker_run-01_eeg-edf.fdt


In [5]:
path_brainvision = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/metrics/sub-02_ses-01_task-checker_run-01_eeg-edf_2_noise_freq_tagging.pkl')
path_sam = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES/GRAD_BCG/sub-02/ses-01/eeg/sub-02_ses-01_task-checker_run-01_eeg_noise_freq_tagging.pkl')
path_sam_asr = Path('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES/GRAD_BCG_ASR/sub-02/ses-01/eeg/sub-02_ses-01_task-checker_run-01_eeg_noise_freq_tagging.pkl')

#brain vision metrics
with open(path_brainvision, 'rb') as f:
    brainvision_metrics = pickle.load(f)

#sam metrics
with open(path_sam, 'rb') as f:
    sam_metrics = pickle.load(f)

#sam asr metrics
with open(path_sam_asr, 'rb') as f:
    sam_asr_metrics = pickle.load(f)

In [14]:
brainvision_metrics.__dir__()

['mne_object',
 'info',
 'amplitude',
 'amplitude_peak_frequency',
 'zscore',
 'zscore_peak_frequency',
 'snr',
 'snr_peak_frequency',
 '__module__',
 '__doc__',
 '__init__',
 'calculate_frequency_metrics',
 'calculate_time_metrics',
 '__dict__',
 '__weakref__',
 '__slotnames__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [5]:
metric_path = path.joinpath('metrics', 'errors_log.pkl')
with open(metric_path, 'wb') as f:
    pickle.dump(errors, f)


In [69]:
f = bids.layout.BIDSFile('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES/BCG/sub-22/ses-02/eeg/sub-22_ses-02_task-checkeroff_run-01_eeg.fif')

In [83]:
f = bids.layout.parse_file_entities(f.filename)

/home/slouviot/miniconda3/envs/mne/lib/python3.12/site-packages/bids/layout/models.py:148: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  warnings.warn("The 'extension' entity currently excludes the leading dot ('.'). "


In [84]:
f

{'session': '02',
 'task': 'checkeroff',
 'run': 1,
 'suffix': 'eeg',
 'extension': 'fif'}

In [64]:
import os
filename_path = Path(filename)
base_filename =  os.path.splitext(filename_path)[0]
saving_filename = base_filename + '_metrics.pkl'

In [65]:
saving_filename

'/projects/EEG_FMRI/bids_eeg/BIDS/NEW/DERIVATIVES/GRAD_BCG/sub-21/ses-01/eeg/sub-21_ses-01_task-checker_run-01_eeg_metrics.pkl'

In [12]:
ssvep_freq_tagging.snr_peak_frequency

array([11.98577881, 11.9934082 , 11.9934082 , 11.9934082 , 11.21520996,
       12.35961914, 11.21520996, 11.07788086, 11.19232178, 11.62719727,
       11.14654541, 11.08551025, 12.01629639, 11.08551025, 12.01629639,
       11.9934082 , 11.9934082 , 11.9934082 , 11.9934082 , 11.9934082 ,
       11.23046875, 11.03210449, 11.23046875, 11.10076904, 11.21520996,
       12.30621338, 11.21520996, 12.30621338, 11.97052002, 11.69586182,
       11.04736328, 11.03973389, 11.21520996, 11.51275635, 11.9934082 ,
       11.98577881, 11.9934082 , 12.0010376 , 12.71057129, 11.07025146,
       11.07788086, 11.08551025, 12.01629639, 11.9934082 , 11.06262207,
       11.03973389, 11.09313965, 11.03210449, 11.36016846, 11.61956787,
       12.86315918, 11.14654541, 12.1383667 , 12.71057129, 11.23046875,
       12.78686523, 11.9934082 , 12.2833252 , 12.93945312, 12.24517822,
       11.21520996, 11.29150391, 12.30621338, 11.08551025])

In [10]:
raw = mne.io.read_raw_eeglab('/projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-02_ses-01_task-checker_run-01_eeg-edf_2.set', preload=True)
def eeg_lab_metrics(raw):
    channel_map = cleaner.map_channel_type(raw)
    raw = cleaner.set_channel_types(raw, channel_map)

    raw_time_metrics = signal_metrics.SignalMetrics(raw)
    raw_time_metrics.calculate_time_metrics()

    noise_freq_tagging = signal_metrics.SignalMetrics(raw)
    noise_freq_tagging.calculate_frequency_metrics(frequency_range=(17,20),
                                                                frequency_type="noise")
    ssvep_freq_tagging = signal_metrics.SignalMetrics(raw)
    ssvep_freq_tagging.calculate_frequency_metrics(frequency_range=(11,13),
                                                                frequency_type="signal")
    ecg_epochs = mne.preprocessing.create_ecg_epochs(raw)
    epochs_ecg_metrics = signal_metrics.SignalMetrics(ecg_epochs)
    epochs_ecg_metrics.calculate_time_metrics()

Reading /projects/EEG_FMRI/bids_eeg/BIDS/NEW/PREP_BV_2024/sub-02_ses-01_task-checker_run-01_eeg-edf_2.fdt
Reading 0 ... 66499  =      0.000 ...   265.996 secs...


<class 'mne.io.eeglab.eeglab.RawEEGLAB'>
Using channel ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 2500 samples (10.000 s)

Number of ECG events detected : 218 (average pulse 49 / min.)
Not setting metadata
218 matching events found
No baseline correction applied
Using data from preloaded Raw for 218 events and 251 original time points ...
2 bad epochs dropped


/home/slouviot/01_projects/EEG_quality_assessment/src/EEG_quality_assessment/signal_metrics.py:111: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = self.mne_object.get_data()


In [19]:
trigger_name = GradientRemover.extract_gradient_trigger_name(raw)
gradient_trigger = mne.events_from_annotations(raw, event_id={trigger_name: 0})

Used Annotations descriptions: ['R128']


In [40]:
annotations.onset[2]

4.1

In [46]:
annotations.description

array(['Sync On', 'Sync On', 'Sync On', 'Sync On', 'Sync On', 'Sync On',
       'Sync On', 'Sync On', 'Sync On', 'Sync On', 'Sync On', 'Sync On',
       'Sync On', 'Sync On', 'Sync On', 'Sync On', 'Sync On', 'Sync On',
       'Sync On', 'R128', 'Sync On', 'R', 'R128', 'S 11', 'S 12',
       'Sync On', 'R', 'S 11', 'R', 'S 11', 'S 12', 'R128', 'Sync On',
       'S 11', 'R', 'S 11', 'S 12', 'Sync On', 'S 11', 'R', 'S 11',
       'S 12', 'R128', 'Sync On', 'R', 'S 11', 'R', 'S 11', 'S 12',
       'R128', 'Sync On', 'S 11', 'S 11', 'S 12', 'Sync On', 'R128',
       'S 11', 'R', 'S 11', 'S 12', 'Sync On', 'R128', 'R', 'S 11',
       'condition 12', 'S 11', 'S 12', 'Sync On', 'TEND', 'R128', 'S 11',
       'R', 'S 11', 'S 12', 'R', 'Sync On', 'R128', 'S 11', 'R', 'S 11',
       'S 12', 'R', 'Sync On', 'R128', 'S 11', 'R', 'S 25', 'R',
       'Sync On', 'R128', 'S 26', 'R', 'S 26', 'S 27', 'R', 'Sync On',
       'S 26', 'R128', 'R', 'S 26', 'S 27', 'Sync On', 'R', 'S 26',
       'R128', 'R', 

In [47]:
import numpy as np

In [14]:
ev = mne.events_from_annotations(raw)

Used Annotations descriptions: ['R', 'R128', 'S 10', 'S 11', 'S 12', 'S 25', 'S 26', 'S 27', 'S 99', 'Sync On', 'TEND', 'condition 12']


In [18]:
trigger_name

'R128'